In [11]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

import re

In [6]:
patients = pd.read_csv('../data/pts.csv')

patients.columns = [x.lower().replace(' ', '_') for x in patients.columns]

db = sqlite3.connect('../data/ehr.sqlite')

patients.to_sql('patients', db, if_exists = 'append', index = False) 

299712

In [7]:
admission = pd.read_csv('../data/hos/admissions.csv')

admission.columns = [x.lower().replace(' ', '_') for x in admission.columns]

db = sqlite3.connect('../data/ehr.sqlite')

admission.to_sql('admission', db, if_exists = 'append', index = False) 

431231

In [8]:
diagicd = pd.read_csv('../data/diag_icd.csv')

diagicd.columns = [x.lower().replace(' ', '_') for x in diagicd.columns]

db = sqlite3.connect('../data/ehr.sqlite')

diagicd.to_sql('diagicd', db, if_exists = 'append', index = False) 

4756326

In [ ]:
# subject_id	hadm_id patient

In [13]:
db = sqlite3.connect('../data/hop_team.sqlite')

for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    df_e78 = pd.read_csv('../data/icd/e78.csv')
    dis = chunk.merge(df_e78, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
#     dic_list = []
# for j in range (10):
    
#     dic = {'subject_id': (dis.loc[j, 'subject_id']),
#            'hadm_id' : (dis.loc[j, 'hadm_id'])}
#     # j = 5
#     list_subheading = re.findall('\n(.+):\n', dis.loc[j,'text'])
#     list_subheading = list_subheading+['$']


#     for i in range (len(list_subheading)-1):
#         text_start = re.search(list_subheading[i] + ":\n", dis.loc[j,'text']).end()
#         if i < (len(list_subheading)-2):
#             text_end = re.search(list_subheading[i+1] + ":\n", dis.loc[j,'text']).start()
#         else:
#             text_end = re.search(list_subheading[i+1], dis.loc[j,'text']).start() 
#         text_parsed = dis.loc[j,'text'][text_start:text_end]

#         dic[list_subheading[i]]=text_parsed
#     dic_list.append(dic)


# df_text = pd.DataFrame(dic_list)
# df_text_clean = df_text[['subject_id', 'hadm_id',
#  'Chief Complaint',
#  'Major Surgical or Invasive Procedure',
#  'History of Present Illness',
#  'Past Medical History',
#  'Social History',
#  'Family History',
#  'Physical Exam',
#  'Discharge',
#  'Pertinent Results',
#  'Brief Hospital Course',
#  'Medications on Admission',
#  'Discharge Medications',
#  'Discharge Disposition',
#  'Discharge Diagnosis',
#  'Discharge Condition',
#  'Discharge Instructions',
#  'Followup Instructions']]

    
#     # filter it to the subjectId and hadm_id
    dis.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [15]:
query = """
SELECT * 
FROM text 
WHERE subject_id = '10000084'
"""

with sqlite3.connect('../data/hop_team.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,long_title,icd
0,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...,"Hyperlipidemia, unspecified",E78
